In [1]:
#| default_exp representation.search

In [2]:
#| hide
%load_ext autoreload
%autoreload 2

In [3]:
#| export
import torch, hnswlib, numpy as np, torch.nn.functional as F
from typing import Optional, Union

from xcai.core import *

## IndexSearch

In [7]:
#| export
class IndexSearch:

    def __init__(self, 
                 index:Optional[hnswlib.Index]=None, 
                 space:Optional[str]='cosine', 
                 efc:Optional[int]=200, 
                 m:Optional[int]=16, 
                 efs:Optional[int]=50, 
                 n_bm:Optional[int]=50, 
                 n_threads:Optional[int]=84):
        store_attr('index,space,efc,m,efs,n_bm,n_threads')
        self.data,self.info = None,None

    def build(self, data:Optional[Union[torch.Tensor,np.array]], info:Optional[Union[torch.Tensor,np.array]]=None):
        if info is None: info = np.arange(data.shape[0])
        if data.shape[0] != info.shape[0]: 
            raise ValueError(f'`data`({data.shape[0]}) and `info`({info.shape[0]}) should have same size.')
            
        if isinstance(data, torch.Tensor): data = data.cpu()
        if isinstance(info, torch.Tensor): info = info.cpu()
        
        self.index = hnswlib.Index(space=self.space, dim=data.shape[1])
        self.index.init_index(max_elements=data.shape[0], ef_construction=self.efc, M=self.m)
        self.index.add_items(data, info, num_threads=self.n_threads)
        self.data,self.info = data,info
        self.index.set_ef(self.efs)

    def get_items(self, idx:Optional[Union[torch.Tensor,np.array]]):
        if isinstance(idx, torch.Tensor): idx = idx.cpu()
        return self.data[idx]

    def proc(self, inputs:Optional[Union[torch.Tensor,np.array]], n_bm:Optional[int]=None):
        n_bm = self.n_bm if n_bm is None else n_bm
        n_bm = self.efs if n_bm > self.efs else n_bm
        if isinstance(inputs, torch.Tensor): inputs = inputs.cpu()
        info, sc = self.index.knn_query(inputs, k=n_bm)
        info, sc, ptr = torch.tensor(info.reshape(-1).astype(np.int64)), torch.tensor(sc.reshape(-1)), torch.full((inputs.shape[0],), n_bm)
        return {'info2data_idx':info, 'info2data_score':1.0-sc, 'info2data_data2ptr':ptr}
        

#### Example

In [ ]:
n,dim = 10000,128
data, info = np.float32(np.random.random((n, dim))), torch.arange(n)

data = torch.rand((n, dim)).to('cuda')

In [ ]:
index = IndexSearch()
index.build(data, info)

In [ ]:
output = index.proc(data)

In [ ]:
output['info2data_idx'].shape, output['info2data_score'].shape, output['info2data_data2ptr'].shape

(torch.Size([500000]), torch.Size([500000]), torch.Size([10000]))

In [ ]:
output['info2data_score'][:10]

tensor([1.0000, 0.8515, 0.8511, 0.8475, 0.8436, 0.8381, 0.8370, 0.8361, 0.8355,
        0.8354])

In [ ]:
torch.norm(data, dim=1)

tensor([6.7123, 6.5139, 6.7348,  ..., 6.4729, 6.6765, 6.2345], device='cuda:0')

## BruteForceSearch

In [8]:
#| export
class BruteForceSearch:
    
    def __init__(
        self, 
        index:Optional[torch.Tensor]=None, 
        n_bm:Optional[int]=50,
        normalize:Optional[bool]=True,
    ):
        store_attr('index,n_bm,normalize')
        
    def build(self, data:Optional[torch.Tensor], info:Optional[torch.Tensor]=None):
        if info is not None and data.shape[0] != info.shape[0]: 
            raise ValueError(f'`data`({data.shape[0]}) and `info`({info.shape[0]}) should have same size.')
        self.index = (data, info)

    def get_items(self, idx:Optional[Union[torch.Tensor,np.array]]):
        return self.index[0][idx]
    
    def proc(self, inputs:Optional[torch.Tensor], n_bm:Optional[int]=None):
        store_attr('n_bm', is_none=False)
        index, info = self.index
        
        inputs = F.normalize(inputs, dim=1) if self.normalize else inputs
        inputs = inputs.to(index.device)
        
        n_bm = min(index.shape[0], self.n_bm)
        index = F.normalize(index, dim=1) if self.normalize else index
        
        sc, idx = torch.topk(inputs@index.T, n_bm, dim=1, largest=True)
        if info is None: info = idx
        else: info = info.unsqueeze(0).expand((idx.shape[0],-1)).gather(1, idx)
            
        info, sc, ptr = info.reshape(-1), sc.reshape(-1), torch.full((inputs.shape[0],), n_bm, device=inputs.device)
        return {'info2data_idx':info, 'info2data_score':sc, 'info2data_data2ptr':ptr}
        

### Example

In [ ]:
n,dim = 10000,128
data = torch.rand((n, dim)).to('cuda')

In [ ]:
index = BruteForceSearch()
index.build(data)

In [ ]:
output = index.proc(data)

In [ ]:
output['info2data_score'][:10]

tensor([1.0000, 0.8101, 0.8094, 0.8083, 0.8072, 0.8072, 0.8036, 0.8028, 0.8021,
        0.8014], device='cuda:0')

In [ ]:
index.get_items(torch.tensor([10,20])).norm(dim=1)

tensor([5.6306, 6.3279], device='cuda:0')